# Getting Started

In [7]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

## Cisco Specific Way to Get ACCESS_TOKEN from Circuit

In [2]:
import openai
openai.api_key = '' #@param {type:"string"}
MODEL = 'code-davinci-002' #@param {type:"string"}

In [3]:
# Importing libraries
import openai
import traceback
import requests
import base64

# Open AI version to use
openai.api_type = "azure"
openai.api_version = "2024-12-01-preview"

# Get API_KEY wrapped in token
client_id = '' #@param {type:"string"}
client_secret = '' #@param {type:"string"}

url = "https://id.cisco.com/oauth2/default/v1/token"

payload = "grant_type=client_credentials"
value = base64.b64encode(f"{client_id}:{client_secret}".encode("utf-8")).decode("utf-8")
headers = {
    "Accept": "*/*",
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": f"Basic {value}",
}

token_response = requests.request("POST", url, headers=headers, data=payload)

print(token_response.text)
token_data = token_response.json()
#api_key = token_data.get('access_token')
#print(api_key) # Set in .env file

{"errorCode":"invalid_client","errorSummary":"Invalid value for 'client_id' parameter.","errorLink":"invalid_client","errorId":"oaeWbfV_TqHR1aeReW60vnZFg","errorCauses":[]}


## Cisco Specific Way to Tie APP_KEY with the ACCESS_TOKEN

## Basic Example

Basic prompt example:

In [ ]:
import os
from dotenv import load_dotenv

from openai import AzureOpenAI

# Loading environment variables
load_dotenv()

client = AzureOpenAI(
    azure_endpoint="https://chat-ai.cisco.com",
    api_key=token_data.get('access_token'),
    api_version="2024-12-01-preview"
)

app_key = '' #@param {type:"string"}

response = client.chat.completions.create(
    model="gpt-4o-mini",  # model = "deployment_name".
    messages=[{"role": "user", "content": "What is the capital of France?"}],
    user=f'{{"appkey": "{app_key}"}}',
)

print(response.choices[0].message.content)

The capital of France is Paris.


## Roles

Chat LLMs provides the ability to specific roles. System role can be used to define the high-level instructions or behavior of the system. 

In [10]:
# use system and user roles to guide the model
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a strict Python 3.11 code generator. Return only code in a fenced block.\n"
                "Follow acceptance criteria precisely."
            ),
        },
        {
            "role": "user",
            "content": (
                "Task: Write a function `sum_even(numbers: list[int]) -> int`.\n"
                "Constraints: O(n), no external libs.\n"
                "Acceptance criteria:\n"
                "- Returns sum of even integers only.\n"
                "- Empty list returns 0.\n"
                "- Negative even numbers count.\n"
                "- Include 3 pytest-style tests in the same code block.\n"
            ),
        },
    ],
    user=f'{{"appkey": "{app_key}"}}'
)

#print(response)
print(response.choices[0].message.content)

```python
def sum_even(numbers: list[int]) -> int:
    return sum(num for num in numbers if num % 2 == 0)

# Pytest-style tests
def test_sum_even():
    assert sum_even([1, 2, 3, 4, 5]) == 6
    assert sum_even([-2, -4, 1, 3, 5]) == -6
    assert sum_even([]) == 0
```
